<a href="https://colab.research.google.com/github/jansoe/public-covid-xr-data/blob/main/PRISMAGoogleDatasetSearch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PRISMA Google Dataset Search

In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import auth
auth.authenticate_user()

In [4]:
import gspread
from oauth2client.client import GoogleCredentials

## Identification Step

Data was manually entered into spreadsheet from google datasets search "covid x-ray"

In [5]:
gc = gspread.authorize(GoogleCredentials.get_application_default())
worksheet = gc.open('DatasetScraping').worksheet('GoogleDataset')

rows = worksheet.get_all_values()
datasets = pd.DataFrame.from_records(rows[1:], columns=rows[0])
print(f'Found datasets: {len(datasets)}')

Found datasets: 199


## Screening step

In [6]:
datasets.screening = datasets.screening.replace('', 'PASS')
datasets.screening.value_counts()

PASS                119
no_chestxraydata     73
na                    4
notpublic             3
Name: screening, dtype: int64

Remove all datasets which have not passed screening

In [7]:
screened = datasets[datasets.screening == 'PASS']
screened.shape

(119, 6)

Deduplicate by dataset link

In [14]:
deduplicated = screened.groupby('Link').first().reset_index()
deduplicated.shape

(112, 6)

## Eligibility step

In [15]:
deduplicated.eligibility = deduplicated.eligibility.replace('', 'PASS')
deduplicated.eligibility.value_counts()

remix             58
no_info           38
nocovid            5
PASS               3
synthetic          3
casecollection     3
no_meta            2
Name: eligibility, dtype: int64

#### Final eligibal datasets

In [16]:
deduplicated[deduplicated.eligibility == 'PASS']

,Link,Title,DOI,screening,nickname,eligibility
0,https://bimcv.cipf.es/bimcv-projects/bimcv-cov...,BIMCV-COVID-19+ dataset,10.21227/w3aw-rv39,PASS,bimcv,PASS
17,https://github.com/ari-dasci/OD-covidgr,COVIDGR Dataset,,PASS,covidgr,PASS
21,https://github.com/ml-workgroup/covid-19-image...,COVID-19 Image Repository,,PASS,mlhannover,PASS


#### Honorabale mentions: not eligibal, but datasets of unique origin

In [17]:
deduplicated[deduplicated.eligibility == 'no_meta']

,Link,Title,DOI,screening,nickname,eligibility
4,https://data.mendeley.com/datasets/4n66brtp4j,COVID Chest X-Rays,,PASS,,no_meta
9,https://data.mendeley.com/datasets/fvp7m8c9ss,Chest X-rays of COVID-19 Pandemic Infected Pat...,10.17632/fvp7m8c9ss,PASS,,no_meta
